# Unit Commitment Problem

## 定数

- $T$: 期間
- $M$: 機械の数
- $l_m$: 機械 $m$ の発電量下限
- $u_m$: 機械 $m$ の発電量上限
- $d_t$: 期 $t$ における需要
- $C_m$: 機械 $m$ で $l_m$ だけ発電したときにかかるコスト
- $c_m$: 機械 $m$ で $l_m$ より多く発電したときにかかるコストの"増加率"
- $f_m$: 機械 $m$ の起動コスト

## 変数

- $x_{tm} \in \{0, 1\}$: 期 $t$ で機械 $m$ が稼働したか否か
- $y_{tm} \in \{0, 1\}$: 期 $t$ で機械 $m$ が起動したか否か
- $p_{tm} \in \mathbb{R}$: 期 $t$ における機械 $m$ の発電量

## 制約

- $y_{tm} \geq x_{tm} - x_{t-1,m}$: 稼働状態が $0 \to 1$ のとき $y_{tm}$ は必ず $1$ を取る
- 下記を加えると$y_{tm}$ は起動したか否かを正確に表せるようになるが, 最適化の過程で自然とそうなるので不要
  - $x_{t-1,m}  - x_{tm} + 2 y_{tm} \leq 1$
- $l_m x_{tm} \leq p_{tm} \leq u_m x_{tm}$: $p_{tm}$ が発電量を表せるようにするためのもの
- $\sum_m p_{tm} \geq d_t$: 期 $t$ における需要が満たされるためのもの

## 目的関数

minimize $c_1 + c_2$ where

- $c_1 = \sum_{t, m} f_m y_{tm}$: 起動コストの合計
- $c_2 = \sum_{t, m} \left( C_m x_{tm} + c_m (p_{tm} - l_m x_{tm}) \right)$: 操業コストの合計

In [1]:
from ortools.sat.python import cp_model

In [2]:
ucp = {}

# 発電ユニット数
ucp["NG"] = 10

# 期間
ucp["NP"] = 24

# 発電ユニット ID
ucp["IDG"] = list(range(ucp["NG"]))

# 発電ユニット初期状態
ucp["UINI"] = [1, 1, 1, 1, 0, 0, 0, 1, 0, 1]

# 発電ユニット初期発電量
ucp["PINI"] = [120.441, 76.9009, 173.322, 85.8445, 0, 0, 0, 212.92, 0, 207.676]

# ramp-up limit
ucp["GRADS"] = [
    86.6823,
    35.6044,
    45.7139,
    109.208,
    23.9261,
    64.6405,
    47.24,
    65.8499,
    58.5693,
    81.7535,
]

# ramp-down limit
ucp["GRADB"] = [
    71.8457,
    43.8919,
    45.4814,
    98.3523,
    33.3089,
    88.5963,
    60.4103,
    69.08,
    56.1335,
    56.002,
]

# minimum up time
ucp["TMINON"] = [6, 2, 6, 1, 2, 6, 1, 6, 6, 6]

# number of hours on at init
ucp["TONINI"] = [4, 2, 2, 3, 0, 0, 0, 4, 0, 1]

# minimum down time
ucp["TMINOFF"] = [6, 3, 6, 1, 3, 6, 1, 6, 6, 6]

# number of hours off at init
ucp["TOFFINI"] = [0, 0, 0, 0, 4, 6, 2, 0, 4, 0]

# cuadratic(←quadratic?) coefficient of generation cost
ucp["A"] = [
    0.000174783,
    0.0613674,
    0.0151426,
    0.0964755,
    0.0400951,
    0.0197901,
    0.067321,
    0.00114823,
    0.0205957,
    0.0163388,
]

# linear coefficient of generation cost
ucp["B"] = [
    7.80793,
    7.36637,
    7.60582,
    4.08036,
    7.00231,
    7.44359,
    4.95782,
    7.90146,
    7.46767,
    7.88726,
]

# fixed part of generation cost
ucp["C"] = [
    525.761,
    327.753,
    516.211,
    127.007,
    264.739,
    462.331,
    139.369,
    535.882,
    420.678,
    549.345,
]

# minimum power
ucp["PMIN"] = [
    98.7759,
    57.64,
    84.1792,
    39.6994,
    69.2816,
    75.2672,
    39.1061,
    95.0505,
    92.4741,
    70.8661,
]

# maximum power
ucp["PMAX"] = [
    294.928,
    166.773,
    210.248,
    110.491,
    162.807,
    308.251,
    103.998,
    322.494,
    280.07,
    246.708,
]

# shutdown cost
ucp["SDCOST"] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# startup power
ucp["PSU"] = [
    98.7759,
    57.64,
    84.1792,
    39.6994,
    69.2816,
    75.2672,
    39.1061,
    95.0505,
    92.4741,
    70.8661,
]

# suthdown power
ucp["PSD"] = [
    98.7759,
    57.64,
    84.1792,
    39.6994,
    69.2816,
    75.2672,
    39.1061,
    95.0505,
    92.4741,
    70.8661,
]

# number of startup cost stages
ucp["NSUC"] = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

# startup cost pairs (SUTIME, SUCOST)
ucp["SUTIME"] = [
    (6, 10),
    (3, 6),
    (6, 11),
    (1, 6),
    (3, 7),
    (6, 10),
    (1, 5),
    (6, 10),
    (6, 10),
    (6, 10),
]
ucp["SUCOST"] = [
    (81140.7, 81348.504),
    (38524.7, 38761.092),
    (56204.7, 56421.415),
    (33103.9, 33334.425),
    (50536.5, 50779.15),
    (76558.6, 76796.432),
    (27774.9, 28019.565),
    (89018.8, 89257.995),
    (60145, 60372.644),
    (75921.2, 76167.968),
]

# hourly demand
ucp["DEMANDA"] = [
    921.901,
    618.645,
    439.931,
    240.88,
    315.435,
    487.5,
    705.024,
    815.824,
    1017.36,
    981.376,
    1048.13,
    1118.54,
    1077,
    575.177,
    763.743,
    1012.73,
    1043.03,
    1121.98,
    1011.22,
    1178.96,
    982.848,
    576.793,
    378.846,
    209.695,
]

# hourly reserve level
ucp["RESERVA"] = [
    99.7616729,
    58.657135,
    39.8198705,
    20.1489616,
    31.0099102,
    45.297915,
    74.7128033,
    51.5114537,
    67.8504853,
    62.5544764,
    79.4700551,
    84.8751508,
    111.898146,
    56.9431557,
    84.0048563,
    62.6151717,
    110.332756,
    99.7194506,
    80.55692,
    126.462323,
    87.4901804,
    47.187493,
    26.1929578,
    20.0154507,
]

# ucp

In [3]:
model = cp_model.CpModel()

terms = list(range(ucp["NP"] + 1))
machines = ucp["IDG"]
base = 1000

# 変数 x[t][m]: 期 t に発電機 m を動かすか否か
x = [[model.new_bool_var(f"x(t={t}, m={m})") for m in machines] for t in terms]
for m in machines:
    # model.add(x[0][m] == ucp["UINI"][m])
    model.add(x[0][m] == 0)  # 初期状態は全て電源オフであるとする

# 変数 p[t][m]: 期 t で発電機 m が発電する電力量(をスケールして整数化)
p = [
    [
        model.new_int_var(0, int(ucp["PMAX"][m] * base), f"p(t={t}, m={m})")
        for m in machines
    ]
    for t in terms
]
for t in terms:
    for m in machines:
        lb_power = int(ucp["PMIN"][m] * base)
        ub_power = int(ucp["PMAX"][m] * base)
        model.add(p[t][m] >= lb_power * x[t][m])
        model.add(p[t][m] <= ub_power * x[t][m])

# 変数 y[t][m]: 期 t で発電機 m が起動したか
y = [[model.new_bool_var(f"y(t={t}, m={m})") for m in machines] for t in terms]
for t in terms:
    for m in machines:
        if t == 0:
            model.add(y[0][m] == 0)
        else:
            model.add(y[t][m] >= x[t][m] - x[t - 1][m])
            # model.add(x[t-1][m] - x[t][m] + 2 * y[t][m] <= 1)

# 需要制約
for t in terms:
    if t == 0:
        continue
    demand = int(ucp["DEMANDA"][t - 1] * base)
    model.add(sum(p[t][m] for m in machines) >= demand)

# 起動コスト
cs = (
    sum(int(ucp["SUCOST"][m][0] * base) * y[t][m] for m in machines for t in terms)
    * base
)

# 操業コスト
cc = 0
for t in terms:
    for m in machines:
        base_cost = int(ucp["C"][m] * base)
        continue_cost = int(ucp["B"][m] * base)
        l_m = int(ucp["PMIN"][m] * base)
        # cc += base_cost * x[t][m] * base + continue_cost * (p[t][m] - l_m * x[t][m])
        cc += (
            base_cost * x[t][m] * base + continue_cost * p[t][m]
        )  # Ax^2 + Bx + C ならこっちが正しいか...?

model.minimize(cs + cc)

In [4]:
solver = cp_model.CpSolver()

# solver.parameters.log_search_progress = True
solver.parameters.max_time_in_seconds = 300

status = solver.solve(model)

In [5]:
print("Statistics")
print(f"  status    : {solver.status_name(status)}")
print(f"  conflicts : {solver.num_conflicts}")
print(f"  branches  : {solver.num_branches}")
print(f"  wall time : {solver.wall_time} s")
if status == cp_model.OPTIMAL or cp_model.FEASIBLE:
    print(f"  total cost: {solver.objective_value / base**2}")

Statistics
  status    : OPTIMAL
  conflicts : 1159
  branches  : 13690
  wall time : 0.44409400000000004 s
  total cost: 456812.750053


In [6]:
print(
    " " * 10
    + "0<"
    + "-" * (len(terms) // 2 - 3)
    + "Term"
    + "-" * (len(terms) // 2 - 4)
    + ">"
    + str(ucp["NP"])
)
for m in machines:
    print(f"Machine {m}", end=" ")
    for t in terms:
        print(f"{"■" if solver.boolean_value(x[t][m]) else "□"}", end="")
    print()

          0<---------Term-------->24
Machine 0 □□□□□□□□□□□□□□□□□□□□□□□□□
Machine 1 □■■■■■■■■■■■■■■■■■■■■□□□□
Machine 2 □□□□□□□□□■■■■■■■■■■■■■□□□
Machine 3 □■■■■■■■■■■■■■■■■■■■■■■■■
Machine 4 □□□□□□□□□□□□□□□□□□□□□□□□□
Machine 5 □■■■■■■■■■■■■■■■■■■■■■■□□
Machine 6 □■■■■■■■■■■■■■■■■■■■■■■■■
Machine 7 □□□□□□□□□□□□□□□□□□□□□□□□□
Machine 8 □■■■■■■■■■■■■■■■■■■■■■■■□
Machine 9 □□□□□□□□□□□□□□□□□□□□□□□□□
